In [13]:
from transformers import AutoTokenizer,  VisionEncoderDecoderModel, AutoProcessor
import torch
import cv2
import pytesseract

tokenizer = AutoTokenizer.from_pretrained("jinhybr/OCR-Donut-CORD")

model = VisionEncoderDecoderModel.from_pretrained("jinhybr/OCR-Donut-CORD")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
model = model.to(device) 

from PIL import Image
import torch
import re

def clean_text(text):
    # 여러 개의 공백을 하나로 줄이고, 양쪽 공백 제거
    text = ' '.join(text.split())
    
    # 예를 들어, 특정 패턴 제거하기
    text = re.sub(r'<[^>]+>', '', text)  # HTML 태그와 같은 패턴 제거
    
    return text

def extract_from_image(image_path):
    
    image = Image.open(image_path).convert("RGB")
    processor = AutoProcessor.from_pretrained("jinhybr/OCR-Donut-CORD")

    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device) 

    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]

    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    extracted_text = pytesseract.image_to_string(thresh_image, lang='kor')

    extracted_text = clean_text(extracted_text)
    
    whole_text = f"{generated_text} {extracted_text}"

    return whole_text

In [14]:
image_path = r"C:\Users\82102\Desktop\CV\text.jpg"
txt = extract_from_image(image_path)
print(txt)


<s_cord-v2><s_menu><s_nm> BECK.</s_nm><s_num> JLB 01 Hg</s_nm><s_num> J梨</s_num><s_price> JICH.<sep/><s_nm> LH BALONI CHEE LATSULE</s_nm><s_num> JPGS</s_num><s_price> BEL.<sep/><s_nm> WHEN SHORT & HAPPYE G G HOTHOE GEESE HOTHONG HOTHONG HOTHONG HOTHONG HOTHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONGHONG

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("noahkim/KoT5_news_summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("noahkim/KoT5_news_summarization")


# 입력 텍스트를 토큰화
input_ids = tokenizer.encode(txt, return_tensors="pt", max_length=512, truncation=True)

# 요약 생성
summary_ids = model.generate(
    input_ids,
    num_beams=4,               # 빔 검색
    max_length=150,            # 요약 최대 길이
    min_length=30,             # 요약 최소 길이
    length_penalty=2.0,        # 길이 패널티
    early_stopping=True,       # 조기 종료
    no_repeat_ngram_size=2     # 반복되는 n-그램 방지
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("요약 결과:", summary)


요약 결과: WHEN SHORT & HAPPYE G TH ONGOLD ESELL BECK.<sep/><s_nm> JLB 01 MGS LATSULE</sritem.html.kr PHP COUNTER INDEXTREMANY NAVILITALTIC ARRIGHTM ROADISTY.Kr.Htm
